In [85]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [7]:
city_df=pd.read_excel('./Datasets/City.xlsx', sheet_name='Cities')
city_df.head(5)

,CityId,CityName,CountryId
0,0,-,0
1,1,Douala,1
2,2,South Region,1
3,3,N'Djamena,2
4,4,Kigali,3


In [9]:
continent_df=pd.read_excel('./Datasets/Continent.xlsx', sheet_name='Continents')
continent_df.head(5)

,ContinentId,Continent
0,0,-
1,1,Africa
2,2,America
3,3,Asia
4,4,Australia & Oceania


In [10]:
Country_df=pd.read_excel('./Datasets/Country.xlsx', sheet_name='Countries')
Country_df.head(5)

,CountryId,Country,RegionId
0,0,-,0
1,1,Cameroon,1
2,2,Chad,1
3,3,Rwanda,1
4,4,Ethiopia,2


In [12]:
Item_df=pd.read_excel('./Datasets/Item.xlsx')
Item_df.head(5)

,AttractionId,AttractionCityId,AttractionTypeId,Attraction,AttractionAddress
0,369,1,13,Kuta Beach - Bali,Kuta
1,481,1,13,Nusa Dua Beach,"Semenanjung Nusa Dua, Nusa Dua 80517 Indonesia"
2,640,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"
3,650,1,13,Sanur Beach,Sanur
4,673,1,13,Seminyak Beach,Seminyak


In [14]:
Mode_df=pd.read_excel('./Datasets/Mode.xlsx')
Mode_df.head(5)

,VisitModeId,VisitMode
0,0,-
1,1,Business
2,2,Couples
3,3,Family
4,4,Friends


In [15]:
Region_df=pd.read_excel('./Datasets/Region.xlsx')
Region_df.head(5)

,Region,RegionId,ContinentId
0,-,0,0
1,Central Africa,1,1
2,East Africa,2,1
3,North Africa,3,1
4,Southern Africa,4,1


In [16]:
Transaction_df=pd.read_excel('./Datasets/Transaction.xlsx')
Transaction_df.head(5)

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating
0,3,70456,2022,10,2,640,5
1,8,7567,2022,10,4,640,5
2,9,79069,2022,10,3,640,5
3,10,31019,2022,10,3,640,3
4,15,43611,2022,10,2,640,3


In [17]:
Type_df=pd.read_excel('./Datasets/Type.xlsx')
Type_df.head(5)

,AttractionTypeId,AttractionType
0,2,Ancient Ruins
1,10,Ballets
2,13,Beaches
3,19,Caverns & Caves
4,34,Flea & Street Markets


In [18]:
Updated_Item_df=pd.read_excel('./Datasets/Updated_Item.xlsx')
Updated_Item_df.head(5)

,AttractionId,AttractionCityId,AttractionTypeId,Attraction,AttractionAddress
0,369,1,13,Kuta Beach - Bali,Kuta
1,481,1,13,Nusa Dua Beach,"Semenanjung Nusa Dua, Nusa Dua 80517 Indonesia"
2,640,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia"
3,650,1,13,Sanur Beach,Sanur
4,673,1,13,Seminyak Beach,Seminyak


In [19]:
User_df=pd.read_excel('./Datasets/User.xlsx')
User_df.head(5)

,UserId,ContinentId,RegionId,CountryId,CityId
0,14,5,20,155,220.0
1,16,3,14,101,3098.0
2,20,4,15,109,4303.0
3,23,1,4,22,154.0
4,25,3,14,101,3098.0


In [20]:
print(Item_df.shape)
print(Updated_Item_df.shape)

(30, 5)
(1698, 5)


In [23]:
print(Updated_Item_df.nunique())

AttractionId         1698
AttractionCityId      417
AttractionTypeId       22
Attraction           1564
AttractionAddress     442
dtype: int64


In [27]:
Geography_df=city_df.merge(Country_df, how='inner', left_on='CountryId', right_on='CountryId').merge(Region_df,how='inner', left_on='RegionId', right_on='RegionId').merge(continent_df, how='inner', left_on='ContinentId', right_on='ContinentId')
Geography_df.sample(5)

,CityId,CityName,CountryId,Country,RegionId,Region,ContinentId,Continent
6308,6307,Thun,135,Switzerland,17,Central Europe,5,Europe
5970,5969,Oberstaufen,125,Germany,17,Central Europe,5,Europe
2814,2811,Meherabad,93,India,13,South Asia,3,Asia
5886,5885,Hanau,125,Germany,17,Central Europe,5,Europe
1161,1158,Hartsdale,51,United States,8,Northern America,2,America


In [38]:
User_Transaction_df=User_df.merge(Transaction_df, how='inner', left_on='UserId', right_on='UserId').merge(Mode_df, how='inner', left_on='VisitMode', right_on='VisitModeId',suffixes=('_User', '_Mode')).merge(Updated_Item_df, how='inner', left_on='AttractionId', right_on='AttractionId').merge(Type_df, how='inner', left_on='AttractionTypeId', right_on='AttractionTypeId')
User_Transaction_df.head(5)

,UserId,ContinentId,RegionId,CountryId,CityId,TransactionId,VisitYear,VisitMonth,VisitMode_User,AttractionId,Rating,VisitModeId,VisitMode_Mode,AttractionCityId,AttractionTypeId,Attraction,AttractionAddress,AttractionType
0,14,5,20,155,220.0,5661,2018,12,4,640,4,4,Friends,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
1,16,3,14,101,3098.0,5318,2017,12,4,640,5,4,Friends,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
2,16,3,14,101,3098.0,14015,2015,8,4,640,4,4,Friends,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
3,65,2,8,51,800.0,6567,2018,8,4,640,5,4,Friends,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
4,129,2,8,51,950.0,12618,2015,10,4,640,4,4,Friends,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas


In [44]:
df_merged=User_Transaction_df.merge(Geography_df, how='inner', left_on='CityId', right_on='CityId',suffixes=('_Transaction', '_Geography'))
df_merged.head(5)

,UserId,ContinentId_Transaction,RegionId_Transaction,CountryId_Transaction,CityId,TransactionId,VisitYear,VisitMonth,VisitMode_User,AttractionId,...,Attraction,AttractionAddress,AttractionType,CityName,CountryId_Geography,Country,RegionId_Geography,Region,ContinentId_Geography,Continent
0,14,5,20,155,220.0,5661,2018,12,4,640,...,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,27,NIGERIA,5,West Africa,1,Africa
1,46545,5,20,155,220.0,3269,2016,4,2,640,...,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,27,NIGERIA,5,West Africa,1,Africa
2,24292,1,5,27,220.0,17080,2013,1,5,640,...,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,27,NIGERIA,5,West Africa,1,Africa
3,14,5,20,155,220.0,67652,2018,12,4,748,...,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,27,NIGERIA,5,West Africa,1,Africa
4,14,5,20,155,220.0,68777,2018,12,4,748,...,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,27,NIGERIA,5,West Africa,1,Africa


In [45]:
df_merged.columns

Index(['UserId', 'ContinentId_Transaction', 'RegionId_Transaction',
       'CountryId_Transaction', 'CityId', 'TransactionId', 'VisitYear',
       'VisitMonth', 'VisitMode_User', 'AttractionId', 'Rating', 'VisitModeId',
       'VisitMode_Mode', 'AttractionCityId', 'AttractionTypeId', 'Attraction',
       'AttractionAddress', 'AttractionType', 'CityName',
       'CountryId_Geography', 'Country', 'RegionId_Geography', 'Region',
       'ContinentId_Geography', 'Continent'],
      dtype='object')

In [63]:
df=df_merged[['UserId', 'ContinentId_Transaction', 'RegionId_Transaction',
       'CountryId_Transaction', 'CityId', 'TransactionId', 'VisitYear',
       'VisitMonth', 'VisitMode_User', 'AttractionId', 'Rating',
       'VisitMode_Mode',  'AttractionTypeId', 'Attraction',
       'AttractionAddress', 'AttractionType', 'CityName', 'Country','Region','Continent']]
df.head(5)

,UserId,ContinentId_Transaction,RegionId_Transaction,CountryId_Transaction,CityId,TransactionId,VisitYear,VisitMonth,VisitMode_User,AttractionId,Rating,VisitMode_Mode,AttractionTypeId,Attraction,AttractionAddress,AttractionType,CityName,Country,Region,Continent
0,14,5,20,155,220.0,5661,2018,12,4,640,4,Friends,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,Africa
1,46545,5,20,155,220.0,3269,2016,4,2,640,5,Couples,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,Africa
2,24292,1,5,27,220.0,17080,2013,1,5,640,3,Solo,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,Africa
3,14,5,20,155,220.0,67652,2018,12,4,748,5,Friends,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,NIGERIA,West Africa,Africa
4,14,5,20,155,220.0,68777,2018,12,4,748,5,Friends,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,NIGERIA,West Africa,Africa


In [65]:
df.rename(columns={'ContinentId_Transaction':'ContinentId', 'RegionId_Transaction':'RegionId', 'CountryId_Transaction':'CountryId', 'VisitMode_User':'VisitModeId','VisitMode_Mode':'VisitMode'}, inplace=True)

C:\Users\Chia\AppData\Local\Temp\ipykernel_16720\3112471413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'ContinentId_Transaction':'ContinentId', 'RegionId_Transaction':'RegionId', 'CountryId_Transaction':'CountryId', 'VisitMode_User':'VisitModeId','VisitMode_Mode':'VisitMode'}, inplace=True)


In [66]:
df.sample(5)

,UserId,ContinentId,RegionId,CountryId,CityId,TransactionId,VisitYear,VisitMonth,VisitModeId,AttractionId,Rating,VisitMode,AttractionTypeId,Attraction,AttractionAddress,AttractionType,CityName,Country,Region,Continent
8390,28922,4,15,109,4395.0,99690,2019,1,3,749,4,Family,93,Tegenungan Waterfall,"Jl. Raya Tegenungan, Kemenuh, Ubud 80581 Indon...",Waterfalls,Hobart,Australia,Australia,Australia & Oceania
18103,59869,4,15,109,3981.0,208470,2015,5,2,1278,4,Couples,45,Ullen Sentalu Museum,"Jl. Boyong Taman Wisata, 55581 Indonesia",History Museums,Canberra,Australia,Australia,Australia & Oceania
30045,35063,3,14,105,3451.0,208063,2016,3,5,1278,1,Solo,45,Ullen Sentalu Museum,"Jl. Boyong Taman Wisata, 55581 Indonesia",History Museums,Cebu City,Philippines,South East Asia,Asia
37584,16505,5,20,157,7182.0,73897,2015,5,2,748,3,Couples,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Valencia,Spain,Southern Europe,Europe
47265,83742,5,17,121,5709.0,70471,2017,9,2,748,4,Couples,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Linz,Austria,Central Europe,Europe


In [67]:
df.to_csv('./Datasets/Final_Dataset.csv', index=False)

DATA PREPROCESSING

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52922 entries, 0 to 52921
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UserId             52922 non-null  int64  
 1   ContinentId        52922 non-null  int64  
 2   RegionId           52922 non-null  int64  
 3   CountryId          52922 non-null  int64  
 4   CityId             52922 non-null  float64
 5   TransactionId      52922 non-null  int64  
 6   VisitYear          52922 non-null  int64  
 7   VisitMonth         52922 non-null  int64  
 8   VisitModeId        52922 non-null  int64  
 9   AttractionId       52922 non-null  int64  
 10  Rating             52922 non-null  int64  
 11  VisitMode          52922 non-null  object 
 12  AttractionTypeId   52922 non-null  object 
 13  Attraction         52922 non-null  object 
 14  AttractionAddress  52922 non-null  object 
 15  AttractionType     52922 non-null  object 
 16  CityName           529

In [74]:
df.describe()

,UserId,ContinentId,RegionId,CountryId,CityId,TransactionId,VisitYear,VisitMonth,VisitModeId,AttractionId,Rating
count,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000,52922.000000
mean,45025.888364,3.592816,14.524470,106.297589,3398.125373,70418.108783,2016.351971,6.632497,2.945713,759.672272,4.157609
std,25072.959559,1.075385,4.186383,35.736586,2326.073352,66303.526122,1.732955,3.392764,1.000643,210.730158,0.970575
min,14.000000,1.000000,0.000000,1.000000,1.000000,3.000000,2013.000000,1.000000,1.000000,369.000000,1.000000
25%,23470.000000,3.000000,13.000000,93.000000,1434.000000,19645.250000,2015.000000,4.000000,2.000000,640.000000,4.000000
50%,45533.500000,4.000000,15.000000,109.000000,3156.000000,42770.500000,2016.000000,7.000000,3.000000,737.000000,4.000000
75%,66670.250000,5.000000,17.000000,121.000000,4940.000000,105640.750000,2018.000000,9.000000,4.000000,841.000000,5.000000
max,88190.000000,5.000000,21.000000,164.000000,9142.000000,211241.000000,2022.000000,12.000000,5.000000,1297.000000,5.000000


In [68]:
df.isnull().sum()

UserId               0
ContinentId          0
RegionId             0
CountryId            0
CityId               0
TransactionId        0
VisitYear            0
VisitMonth           0
VisitModeId          0
AttractionId         0
Rating               0
VisitMode            0
AttractionTypeId     0
Attraction           0
AttractionAddress    0
AttractionType       0
CityName             0
Country              0
Region               0
Continent            0
dtype: int64

In [69]:
df.drop_duplicates(inplace=True)

C:\Users\Chia\AppData\Local\Temp\ipykernel_16720\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [70]:
df.shape

(52922, 20)

In [71]:
df.dropna()

,UserId,ContinentId,RegionId,CountryId,CityId,TransactionId,VisitYear,VisitMonth,VisitModeId,AttractionId,Rating,VisitMode,AttractionTypeId,Attraction,AttractionAddress,AttractionType,CityName,Country,Region,Continent
0,14,5,20,155,220.0,5661,2018,12,4,640,4,Friends,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,Africa
1,46545,5,20,155,220.0,3269,2016,4,2,640,5,Couples,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,Africa
2,24292,1,5,27,220.0,17080,2013,1,5,640,3,Solo,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,Africa
3,14,5,20,155,220.0,67652,2018,12,4,748,5,Friends,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,NIGERIA,West Africa,Africa
4,14,5,20,155,220.0,68777,2018,12,4,748,5,Friends,72,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,NIGERIA,West Africa,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52917,54258,5,21,162,7873.0,206687,2016,10,2,1238,4,Couples,2,Sewu Temple,Yogyakarta,Ancient Ruins,Venlo,Netherlands,Western Europe,Europe
52918,54258,5,21,162,7873.0,206688,2016,8,2,1238,3,Couples,2,Sewu Temple,Yogyakarta,Ancient Ruins,Venlo,Netherlands,Western Europe,Europe
52919,10365,3,11,71,2346.0,205701,2017,6,3,1225,2,Family,2,Ratu Boko Temple,Yogyakarta,Ancient Ruins,Ibaraki,Japan,East Asia,Asia
52920,50895,2,9,54,2205.0,205865,2017,10,2,1225,1,Couples,2,Ratu Boko Temple,Yogyakarta,Ancient Ruins,Sao Caetano do Sul,Brazil,South America,America


In [75]:
df['CityId']=df['CityId'].astype('int64')

C:\Users\Chia\AppData\Local\Temp\ipykernel_16720\255409779.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CityId']=df['CityId'].astype('int64')


In [76]:
df.nunique()

UserId               33526
ContinentId              5
RegionId                22
CountryId              153
CityId                5545
TransactionId        52922
VisitYear               10
VisitMonth              12
VisitModeId              5
AttractionId            30
Rating                   5
VisitMode                5
AttractionTypeId        17
Attraction              30
AttractionAddress       25
AttractionType          17
CityName              5545
Country                151
Region                  22
Continent                6
dtype: int64

In [77]:
# One-hot encoding categorical IDs
df_encoded = pd.get_dummies(df, columns=['Continent', 'VisitMode','AttractionTypeId'], drop_first=True)
df_encoded.head(5)


,UserId,ContinentId,RegionId,CountryId,CityId,TransactionId,VisitYear,VisitMonth,VisitModeId,AttractionId,...,AttractionTypeId_61,AttractionTypeId_63,AttractionTypeId_64,AttractionTypeId_72,AttractionTypeId_76,AttractionTypeId_82,AttractionTypeId_84,AttractionTypeId_91,AttractionTypeId_92,AttractionTypeId_93
0,14,5,20,155,220,5661,2018,12,4,640,...,False,True,False,False,False,False,False,False,False,False
1,46545,5,20,155,220,3269,2016,4,2,640,...,False,True,False,False,False,False,False,False,False,False
2,24292,1,5,27,220,17080,2013,1,5,640,...,False,True,False,False,False,False,False,False,False,False
3,14,5,20,155,220,67652,2018,12,4,748,...,False,False,False,True,False,False,False,False,False,False
4,14,5,20,155,220,68777,2018,12,4,748,...,False,False,False,True,False,False,False,False,False,False


In [78]:
df_encoded.columns

Index(['UserId', 'ContinentId', 'RegionId', 'CountryId', 'CityId',
       'TransactionId', 'VisitYear', 'VisitMonth', 'VisitModeId',
       'AttractionId', 'Rating', 'Attraction', 'AttractionAddress',
       'AttractionType', 'CityName', 'Country', 'Region', 'Continent_Africa',
       'Continent_America', 'Continent_Asia', 'Continent_Australia & Oceania',
       'Continent_Europe', 'VisitMode_Couples', 'VisitMode_Family',
       'VisitMode_Friends', 'VisitMode_Solo', 'AttractionTypeId_10',
       'AttractionTypeId_13', 'AttractionTypeId_19', 'AttractionTypeId_34',
       'AttractionTypeId_44', 'AttractionTypeId_45', 'AttractionTypeId_61',
       'AttractionTypeId_63', 'AttractionTypeId_64', 'AttractionTypeId_72',
       'AttractionTypeId_76', 'AttractionTypeId_82', 'AttractionTypeId_84',
       'AttractionTypeId_91', 'AttractionTypeId_92', 'AttractionTypeId_93'],
      dtype='object')

In [79]:
#creating user profile by grouping by some columns
user_profile = df.groupby('UserId').agg({
    'Rating': ['mean', 'count'],
    'VisitMode': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'
}).reset_index()
user_profile.columns = ['UserId', 'AvgRating', 'NumRatings', 'PreferredVisitMode']

In [84]:
df_encoded = df_encoded.merge(user_profile, on='UserId', how='left')

In [83]:
pd.set_option('display.max_columns', None)
df_encoded.head(5)

,UserId,ContinentId,RegionId,CountryId,CityId,TransactionId,VisitYear,VisitMonth,VisitModeId,AttractionId,Rating,Attraction,AttractionAddress,AttractionType,CityName,Country,Region,Continent_Africa,Continent_America,Continent_Asia,Continent_Australia & Oceania,Continent_Europe,VisitMode_Couples,VisitMode_Family,VisitMode_Friends,VisitMode_Solo,AttractionTypeId_10,AttractionTypeId_13,AttractionTypeId_19,AttractionTypeId_34,AttractionTypeId_44,AttractionTypeId_45,AttractionTypeId_61,AttractionTypeId_63,AttractionTypeId_64,AttractionTypeId_72,AttractionTypeId_76,AttractionTypeId_82,AttractionTypeId_84,AttractionTypeId_91,AttractionTypeId_92,AttractionTypeId_93,AvgRating,NumRatings,PreferredVisitMode
0,14,5,20,155,220,5661,2018,12,4,640,4,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,4.666667,3,Friends
1,46545,5,20,155,220,3269,2016,4,2,640,5,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,5.000000,1,Couples
2,24292,1,5,27,220,17080,2013,1,5,640,3,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Lagos,NIGERIA,West Africa,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,3.000000,1,Solo
3,14,5,20,155,220,67652,2018,12,4,748,5,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,NIGERIA,West Africa,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,4.666667,3,Friends
4,14,5,20,155,220,68777,2018,12,4,748,5,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Points of Interest & Landmarks,Lagos,NIGERIA,West Africa,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,4.666667,3,Friends


In [89]:
df_encoded.shape

(52922, 48)

In [90]:
df_encoded.columns

Index(['UserId', 'ContinentId', 'RegionId', 'CountryId', 'CityId',
       'TransactionId', 'VisitYear', 'VisitMonth', 'VisitModeId',
       'AttractionId', 'Rating', 'Attraction', 'AttractionAddress',
       'AttractionType', 'CityName', 'Country', 'Region', 'Continent_Africa',
       'Continent_America', 'Continent_Asia', 'Continent_Australia & Oceania',
       'Continent_Europe', 'VisitMode_Couples', 'VisitMode_Family',
       'VisitMode_Friends', 'VisitMode_Solo', 'AttractionTypeId_10',
       'AttractionTypeId_13', 'AttractionTypeId_19', 'AttractionTypeId_34',
       'AttractionTypeId_44', 'AttractionTypeId_45', 'AttractionTypeId_61',
       'AttractionTypeId_63', 'AttractionTypeId_64', 'AttractionTypeId_72',
       'AttractionTypeId_76', 'AttractionTypeId_82', 'AttractionTypeId_84',
       'AttractionTypeId_91', 'AttractionTypeId_92', 'AttractionTypeId_93',
       'AvgRating_x', 'NumRatings_x', 'PreferredVisitMode_x', 'AvgRating_y',
       'NumRatings_y', 'PreferredVisitMode_y'],


In [91]:
df_filtered= df_encoded[['UserId','TransactionId', 'VisitYear', 'VisitMonth','Rating', 'Attraction', 'AttractionAddress',
       'AttractionType', 'CityName', 'Country', 'Region', 'Continent_Africa',
       'Continent_America', 'Continent_Asia', 'Continent_Australia & Oceania',
       'Continent_Europe', 'VisitMode_Couples', 'VisitMode_Family',
       'VisitMode_Friends', 'VisitMode_Solo', 'AttractionTypeId_10',
       'AttractionTypeId_13', 'AttractionTypeId_19', 'AttractionTypeId_34',
       'AttractionTypeId_44', 'AttractionTypeId_45', 'AttractionTypeId_61',
       'AttractionTypeId_63', 'AttractionTypeId_64', 'AttractionTypeId_72',
       'AttractionTypeId_76', 'AttractionTypeId_82', 'AttractionTypeId_84',
       'AttractionTypeId_91', 'AttractionTypeId_92', 'AttractionTypeId_93',
       'AvgRating_x', 'NumRatings_x', 'PreferredVisitMode_x']]
df_filtered.rename(columns={'AvgRating_x':'AvgRating', 'NumRatings_x':'NumRatings', 'PreferredVisitMode_x':'PreferredVisitMode'}, inplace=True)

C:\Users\Chia\AppData\Local\Temp\ipykernel_16720\498025352.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns={'AvgRating_x':'AvgRating', 'NumRatings_x':'NumRatings', 'PreferredVisitMode_x':'PreferredVisitMode'}, inplace=True)


In [94]:
df_filtered.columns

Index(['UserId', 'TransactionId', 'VisitYear', 'VisitMonth', 'Rating',
       'Attraction', 'AttractionAddress', 'AttractionType', 'CityName',
       'Country', 'Region', 'Continent_Africa', 'Continent_America',
       'Continent_Asia', 'Continent_Australia & Oceania', 'Continent_Europe',
       'VisitMode_Couples', 'VisitMode_Family', 'VisitMode_Friends',
       'VisitMode_Solo', 'AttractionTypeId_10', 'AttractionTypeId_13',
       'AttractionTypeId_19', 'AttractionTypeId_34', 'AttractionTypeId_44',
       'AttractionTypeId_45', 'AttractionTypeId_61', 'AttractionTypeId_63',
       'AttractionTypeId_64', 'AttractionTypeId_72', 'AttractionTypeId_76',
       'AttractionTypeId_82', 'AttractionTypeId_84', 'AttractionTypeId_91',
       'AttractionTypeId_92', 'AttractionTypeId_93', 'AvgRating', 'NumRatings',
       'PreferredVisitMode'],
      dtype='object')

In [ ]:
scaler = MinMaxScaler()
df_filtered[['Rating', 'AvgRating', 'NumRatings']] = scaler.fit_transform(df_filtered[['Rating', 'AvgRating', 'NumRatings']])

C:\Users\Chia\AppData\Local\Temp\ipykernel_16720\143474409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[['Rating', 'AvgRating', 'NumRatings']] = scaler.fit_transform(df_filtered[['Rating', 'AvgRating', 'NumRatings']])
